In [16]:
%pip install -q dvc==3.53.2 joblib matplotlib 

from pathlib import Path
ROOT = Path.cwd() if Path.cwd().name=="credit" else Path.cwd().parent
ROOT


Note: you may need to restart the kernel to use updated packages.


WindowsPath('C:/Users/USER/Desktop/credit')

# Дерево папок + сырой датасет под DVC

In [18]:
import shutil, os

# создадим нужные папки
for p in ["data/raw", "data/processed", "models", "src/data", "src/models"]:
    (ROOT / p).mkdir(parents=True, exist_ok=True)

# положи исходный CSV в data/raw (если его там нет)
raw_csv = ROOT / "data" / "raw" / "UCI_Credit_Card.csv"
if not raw_csv.exists():
    # если файл лежит рядом с ноутбуком/где-то ещё — поправь путь ниже и скопируй
    possible = ROOT / "UCI_Credit_Card.csv"
    if possible.exists():
        shutil.copy2(possible, raw_csv)

raw_csv, raw_csv.exists()


(WindowsPath('C:/Users/USER/Desktop/credit/data/raw/UCI_Credit_Card.csv'),
 True)

## DVC init + добавим сырой CSV в DVC

In [20]:
import subprocess, sys

def sh(cmd):
    r = subprocess.run(cmd, cwd=ROOT, shell=True, text=True, capture_output=True)
    print(">", cmd, "\n", r.stdout or r.stderr); 
    return r

sh("dvc init -q")
# добавим сырой датасет под контроль DVC (в гит уйдёт .dvc-файл, сам csv — в .gitignore)
sh("dvc add data/raw/UCI_Credit_Card.csv")


> dvc init -q 
 
> dvc add data/raw/UCI_Credit_Card.csv 
 
To track the changes with git, run:

	git add 'data\raw\UCI_Credit_Card.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true



CompletedProcess(args='dvc add data/raw/UCI_Credit_Card.csv', returncode=0, stdout="\nTo track the changes with git, run:\n\n\tgit add 'data\\raw\\UCI_Credit_Card.csv.dvc'\n\nTo enable auto staging, run:\n\n\tdvc config core.autostage true\n", stderr='\\u280b Checking graph\n\n')

## Скрипт src/models/train.py

In [22]:
code = r"""
import json, argparse
from pathlib import Path
import joblib, pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

TARGET = "default.payment.next.month"

def make_preprocess(X):
    all_cols = X.columns.tolist()
    cat = [c for c in ["SEX","EDUCATION","MARRIAGE"] if c in all_cols] + [c for c in all_cols if c.startswith("PAY_")]
    cat = sorted(list(dict.fromkeys(cat)))
    num = [c for c in all_cols if c not in cat]

    num_tf = Pipeline([("imputer", SimpleImputer(strategy="median")),
                       ("scaler", StandardScaler())])
    cat_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                       ("onehot", OneHotEncoder(handle_unknown="ignore"))])
    return ColumnTransformer([("num", num_tf, num), ("cat", cat_tf, cat)])

def main(train_path, test_path, model_out, metrics_out):
    train = pd.read_csv(train_path)
    test  = pd.read_csv(test_path)
    for df in (train, test):
        if "ID" in df.columns: df.drop(columns=["ID"], inplace=True)

    X_train, y_train = train.drop(columns=[TARGET]), train[TARGET]
    X_test,  y_test  = test.drop(columns=[TARGET]),  test[TARGET]

    pipe = Pipeline([
        ("preprocess", make_preprocess(X_train)),
        ("clf", GradientBoostingClassifier(learning_rate=0.1, n_estimators=150, random_state=42))
    ])
    pipe.fit(X_train, y_train)

    y_proba = pipe.predict_proba(X_test)[:,1]
    y_pred  = pipe.predict(X_test)

    metrics = {
        "model": "GradientBoostingClassifier",
        "model_params": {"learning_rate": 0.1, "n_estimators": 150},
        "roc_auc": float(roc_auc_score(y_test, y_proba)),
        "precision": float(precision_score(y_test, y_pred, zero_division=0)),
        "recall": float(recall_score(y_test, y_pred, zero_division=0)),
        "f1": float(f1_score(y_test, y_pred, zero_division=0))
    }

    model_out = Path(model_out); model_out.parent.mkdir(parents=True, exist_ok=True)
    metrics_out = Path(metrics_out); metrics_out.parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(pipe, model_out)
    Path(metrics_out).write_text(json.dumps(metrics, indent=2), encoding="utf-8")

if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("--train", required=True)
    p.add_argument("--test", required=True)
    p.add_argument("--model-out", required=True)
    p.add_argument("--metrics-out", required=True)
    args = p.parse_args()
    main(args.train, args.test, args.model_out, args.metrics_out)
"""
path = ROOT / "src" / "models" / "train.py"
path.write_text(code, encoding="utf-8"); path


WindowsPath('C:/Users/USER/Desktop/credit/src/models/train.py')

## Создаём dvc.yaml (2 стадии)

In [24]:
dvc_yaml = f"""
stages:
  prepare:
    cmd: python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/
    deps:
    - src/data/make_dataset.py
    - data/raw/UCI_Credit_Card.csv
    outs:
    - data/processed/train.csv
    - data/processed/test.csv

  train:
    cmd: python src/models/train.py --train data/processed/train.csv --test data/processed/test.csv --model-out models/credit_default_model.pkl --metrics-out models/metrics.json
    deps:
    - src/models/train.py
    - data/processed/train.csv
    - data/processed/test.csv
    outs:
    - models/credit_default_model.pkl
    metrics:
    - models/metrics.json:
        cache: false
"""
p = (ROOT / "dvc.yaml")
p.write_text(dvc_yaml, encoding="utf-8"); p


WindowsPath('C:/Users/USER/Desktop/credit/dvc.yaml')

In [32]:
# пересоберёт обе стадии, положит артефакты и метрики
sh("dvc repro")
print((ROOT/"models"/"metrics.json").read_text()[:250])


> dvc repro 
 'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

{
  "model": "GradientBoostingClassifier",
  "model_params": {
    "learning_rate": 0.1,
    "n_estimators": 150
  },
  "roc_auc": 0.7705572956671517,
  "precision": 0.6647230320699709,
  "recall": 0.3436322532027129,
  "f1": 0.45305514157973176
}


In [40]:
sh("git add notebooks/04_dvc_pipeline.ipynb")
sh('git commit -m "wip(dvc): update 04_dvc_pipeline notebook"')

sh("git pull --rebase origin main")

sh("git push")


> git add notebooks/04_dvc_pipeline.ipynb 

> git commit -m "wip(dvc): update 04_dvc_pipeline notebook" 
 [main e24e48a] wip(dvc): update 04_dvc_pipeline notebook
 1 file changed, 75 insertions(+), 11 deletions(-)

> git pull --rebase origin main 
 From https://github.com/pero1x1/credit
 * branch            main       -> FETCH_HEAD
Rebasing (1/5)
Rebasing (2/5)
Rebasing (3/5)
Rebasing (4/5)
Rebasing (5/5)
Successfully rebased and updated refs/heads/main.

> git push 
 To https://github.com/pero1x1/credit.git
   a824c4a..f57feeb  main -> main



CompletedProcess(args='git push', returncode=0, stdout='', stderr='To https://github.com/pero1x1/credit.git\n   a824c4a..f57feeb  main -> main\n')

In [42]:
# не забываем requirements и .gitignore обновить, если нужно
req = ROOT/"requirements.txt"
txt = req.read_text(encoding="utf-8") if req.exists() else ""
for line in ["dvc==3.53.2", "joblib"]:
    if line not in txt:
        txt += ("" if txt.endswith("\n") else "\n") + line + "\n"
req.write_text(txt, encoding="utf-8")

sh("git add -A")
sh('git commit -m "build(dvc): init + dvc.yaml (prepare/train) + data/raw under DVC + reproducible training"')
sh("git push")


> git add -A 

> git commit -m "build(dvc): init + dvc.yaml (prepare/train) + data/raw under DVC + reproducible training" 
 [main 493a750] build(dvc): init + dvc.yaml (prepare/train) + data/raw under DVC + reproducible training
 1 file changed, 20 insertions(+), 90 deletions(-)

> git push 
 To https://github.com/pero1x1/credit.git
   f57feeb..493a750  main -> main



CompletedProcess(args='git push', returncode=0, stdout='', stderr='To https://github.com/pero1x1/credit.git\n   f57feeb..493a750  main -> main\n')

## dvc.yaml + params.yaml 

In [3]:
import pathlib, textwrap, subprocess, json, sys, os, yaml

ROOT = pathlib.Path.cwd() if pathlib.Path.cwd().name=="credit" else pathlib.Path.cwd().parent

def sh(cmd):
    r = subprocess.run(cmd, cwd=ROOT, shell=True, text=True, capture_output=True)
    print(">", cmd, "\n", r.stdout or r.stderr); return r

# --- params.yaml (чтобы train.py мог читать гиперпараметры)
params = {
    "model": {
        "type": "GradientBoostingClassifier",
        "n_estimators": 150,
        "learning_rate": 0.1,
        "random_state": 42
    }
}
(ROOT / "params.yaml").write_text(yaml.safe_dump(params, sort_keys=False), encoding="utf-8")

# --- dvc.yaml (2 стадии: prepare -> train)
dvc_yaml = textwrap.dedent("""
stages:
  prepare:
    cmd: python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/
    deps:
      - src/data/make_dataset.py
      - data/raw/UCI_Credit_Card.csv
    outs:
      - data/processed/train.csv
      - data/processed/test.csv

  train:
    cmd: python src/models/train.py data/processed/train.csv data/processed/test.csv models/
    deps:
      - src/models/train.py
      - data/processed/train.csv
      - data/processed/test.csv
      - params.yaml
    outs:
      - models/credit_default_model.pkl
    metrics:
      - models/metrics.json:
          cache: false
""").strip()
(ROOT / "dvc.yaml").write_text(dvc_yaml, encoding="utf-8")

print((ROOT/"dvc.yaml").read_text()[:400])


stages:
  prepare:
    cmd: python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/
    deps:
      - src/data/make_dataset.py
      - data/raw/UCI_Credit_Card.csv
    outs:
      - data/processed/train.csv
      - data/processed/test.csv

  train:
    cmd: python src/models/train.py data/processed/train.csv data/processed/test.csv models/
    deps:
      - src/models/train.py


In [5]:
# патчим src/models/train.py: чтение params.yaml и запись metrics.json
from pathlib import Path
p = ROOT/"src/models/train.py"
code = p.read_text(encoding="utf-8")

if "yaml.safe_load" not in code:
    patched = """
import sys, json, joblib, pandas as pd
from pathlib import Path
import yaml
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split

def main(train_path, test_path, out_dir):
    params = yaml.safe_load(open('params.yaml'))['model']
    model_type = params.get('type', 'GradientBoostingClassifier')

    X_train = pd.read_csv(train_path).drop(columns=['default.payment.next.month'])
    y_train = pd.read_csv(train_path)['default.payment.next.month']
    X_test  = pd.read_csv(test_path).drop(columns=['default.payment.next.month'])
    y_test  = pd.read_csv(test_path)['default.payment.next.month']

    if model_type == 'RandomForestClassifier':
        model = RandomForestClassifier(
            n_estimators=params.get('n_estimators', 200),
            max_depth=params.get('max_depth', None),
            class_weight=params.get('class_weight', None),
            random_state=params.get('random_state', 42)
        )
    else:
        model = GradientBoostingClassifier(
            n_estimators=params.get('n_estimators', 150),
            learning_rate=params.get('learning_rate', 0.1),
            random_state=params.get('random_state', 42)
        )

    model.fit(X_train, y_train)
    proba = model.predict_proba(X_test)[:,1] if hasattr(model,'predict_proba') else model.decision_function(X_test)
    pred  = (proba >= 0.5).astype(int)

    metrics = {
        "roc_auc": float(roc_auc_score(y_test, proba)),
        "precision": float(precision_score(y_test, pred, zero_division=0)),
        "recall": float(recall_score(y_test, pred, zero_division=0)),
        "f1": float(f1_score(y_test, pred, zero_division=0)),
        "model": model.__class__.__name__,
        "model_params": {k: getattr(model, k, None) for k in ['n_estimators','learning_rate','max_depth','class_weight']}
    }

    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    joblib.dump(model, out/'credit_default_model.pkl')
    (out/'metrics.json').write_text(json.dumps(metrics, indent=2), encoding='utf-8')

if __name__ == "__main__":
    train_path, test_path, out_dir = sys.argv[1], sys.argv[2], sys.argv[3]
    main(train_path, test_path, out_dir)
"""
    p.write_text(patched, encoding="utf-8")
    print("train.py обновлён под params.yaml")
else:
    print("train.py уже читает params.yaml — ок")


train.py обновлён под params.yaml


## Первый прогон пайплайна

In [7]:
sh("dvc repro")            # выполнит prepare -> train, создаст dvc.lock
sh("dvc metrics show -T")  # покажет models/metrics.json
sh("dvc dag")              # граф стадий


> dvc repro 
 'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

> dvc metrics show -T 
 Revision    Path                 f1       model                       model_params.learning_rate    model_params.n_estimators    precision    recall    roc_auc
workspace   models\metrics.json  0.45306  GradientBoostingClassifier  0.1                           150                          0.66472      0.34363   0.77056

> dvc dag 
 +----------------------------------+ 
| data\raw\UCI_Credit_Card.csv.dvc | 
+----------------------------------+ 
                  *                  
                  *                  
                  *                  
            +---------+              
            | prepare |              
            +---------+              
                  *                  
                  *                  
                  *                  
         

CompletedProcess(args='dvc dag', returncode=0, stdout='+----------------------------------+ \n| data\\raw\\UCI_Credit_Card.csv.dvc | \n+----------------------------------+ \n                  *                  \n                  *                  \n                  *                  \n            +---------+              \n            | prepare |              \n            +---------+              \n                  *                  \n                  *                  \n                  *                  \n              +-------+              \n              | train |              \n              +-------+              \n', stderr='')

## Коммит и пуш

In [9]:
sh("git add -A")
sh('git commit -m "build(dvc): pipeline prepare/train + first repro; lock + metrics"')
sh("git push")


> git add -A 

> git commit -m "build(dvc): pipeline prepare/train + first repro; lock + metrics" 
 [main 71f91b0] build(dvc): pipeline prepare/train + first repro; lock + metrics
 5 files changed, 204 insertions(+), 69 deletions(-)
 create mode 100644 notebooks/Untitled.ipynb
 create mode 100644 params.yaml

> git push 
 To https://github.com/pero1x1/credit.git
   493a750..71f91b0  main -> main



CompletedProcess(args='git push', returncode=0, stdout='', stderr='To https://github.com/pero1x1/credit.git\n   493a750..71f91b0  main -> main\n')

## DVC Experiments (params + repro + exp show) и локальный remote

1) Перевожу `train.py` на чтение гиперпараметров из `params.yaml`.
2) В `dvc.yaml` подключаю `params` для стадии `train`.
3) Делаю пару экспериментов через `dvc exp run -S ...` и сравниваю метрики.
4) Настраиваю локальный DVC remote (папка `dvcstore/`) и пушу артефакты.


In [22]:
import subprocess, pathlib, sys

ROOT = pathlib.Path.cwd() if pathlib.Path.cwd().name=="credit" else pathlib.Path.cwd().parent

def sh(cmd):
    r = subprocess.run(cmd, cwd=ROOT, shell=True, text=True, capture_output=True)
    print(">", cmd, "\n", r.stdout or r.stderr)
    return r

ROOT


WindowsPath('C:/Users/USER/Desktop/credit')

In [24]:
req = (ROOT / "requirements.txt")
text = req.read_text(encoding="utf-8") if req.exists() else ""
need = ["pyyaml", "dvc==3.53.2"]
for pkg in need:
    if pkg not in text.lower():
        text += ("\n" + pkg + "\n")
req.write_text(text.strip()+"\n", encoding="utf-8")

# локально для ядра
sh(f"{ROOT/' .venv/Scripts/python' if (ROOT/'.venv/Scripts/python').exists() else sys.executable} -m pip install -q pyyaml dvc==3.53.2")


> C:\Users\USER\Desktop\credit\.venv\Scripts\python.exe -m pip install -q pyyaml dvc==3.53.2 
 


CompletedProcess(args='C:\\Users\\USER\\Desktop\\credit\\.venv\\Scripts\\python.exe -m pip install -q pyyaml dvc==3.53.2', returncode=0, stdout='', stderr='')

In [26]:
code = r'''
import argparse
from pathlib import Path
import joblib, pandas as pd
import yaml

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

TARGET = "default.payment.next.month"

def make_preprocess(df):
    all_cols = list(df.columns)
    cat = [c for c in ["SEX","EDUCATION","MARRIAGE"] if c in all_cols] + [c for c in all_cols if c.startswith("PAY_")]
    num = [c for c in all_cols if c not in cat]

    num_tf = Pipeline([("imputer", SimpleImputer(strategy="median")),
                       ("scaler",  StandardScaler())])

    cat_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                       ("onehot",  OneHotEncoder(handle_unknown="ignore"))])

    return ColumnTransformer([("num",  num_tf, num), ("cat",  cat_tf, cat)])

def train(train_path, test_path, model_out, metrics_out):
    # читаю параметры из params.yaml
    P = yaml.safe_load((Path(__file__).parents[1] / "params.yaml").read_text(encoding="utf-8"))
    lr = float(P["model"]["learning_rate"])
    n_est = int(P["model"]["n_estimators"])

    train = pd.read_csv(train_path)
    test  = pd.read_csv(test_path)
    if "ID" in train.columns: train.drop(columns=["ID"], inplace=True)
    if "ID" in test.columns:  test.drop(columns=["ID"],  inplace=True)

    X_train, y_train = train.drop(columns=[TARGET]), train[TARGET]
    X_test,  y_test  = test.drop(columns=[TARGET]),  test[TARGET]

    pipe = Pipeline([
        ("preprocess", make_preprocess(train)),
        ("clf", GradientBoostingClassifier(learning_rate=lr, n_estimators=n_est, random_state=42)),
    ])

    pipe.fit(X_train, y_train)
    y_prob = pipe.predict_proba(X_test)[:, 1]
    y_pred = pipe.predict(X_test)

    metrics = {
        "model": "GradientBoostingClassifier",
        "model_params": {"learning_rate": lr, "n_estimators": n_est},
        "roc_auc":  float(roc_auc_score(y_test, y_prob)),
        "precision": float(precision_score(y_test, y_pred, zero_division=0)),
        "recall": float(recall_score(y_test, y_pred, zero_division=0)),
        "f1": float(f1_score(y_test, y_pred, zero_division=0)),
    }

    Path(model_out).parent.mkdir(parents=True, exist_ok=True)
    Path(metrics_out).parent.mkdir(parents=True, exist_ok=True)
    joblib.dump(pipe, model_out)
    Path(metrics_out).write_text(__import__("json").dumps(metrics, indent=2, ensure_ascii=False), encoding="utf-8")

if __name__ == "__main__":
    ap = argparse.ArgumentParser()
    ap.add_argument("--train", required=True)
    ap.add_argument("--test",  required=True)
    ap.add_argument("--model-out", required=True)
    ap.add_argument("--metrics-out", required=True)
    args = ap.parse_args()
    train(args.train, args.test, args.model_out, args.metrics_out)
'''
p = ROOT / "src" / "models" / "train.py"
p.write_text(code.strip()+"\n", encoding="utf-8")
p


WindowsPath('C:/Users/USER/Desktop/credit/src/models/train.py')

In [28]:
import yaml, io
params_path = ROOT / "params.yaml"
P = {"model": {"learning_rate": 0.10, "n_estimators": 150}}
if params_path.exists():
    P0 = yaml.safe_load(params_path.read_text(encoding="utf-8")) or {}
    # аккуратно обновляю только нужные ключи
    P0.setdefault("model", {})
    P0["model"].setdefault("learning_rate", 0.10)
    P0["model"].setdefault("n_estimators", 150)
    P = P0
params_path.write_text(yaml.safe_dump(P, sort_keys=False, allow_unicode=True), encoding="utf-8")
P


{'model': {'type': 'GradientBoostingClassifier',
  'n_estimators': 150,
  'learning_rate': 0.1,
  'random_state': 42}}

In [30]:
dvc_yaml = (ROOT / "dvc.yaml").read_text(encoding="utf-8")
if "params:" not in dvc_yaml:
    dvc_yaml = dvc_yaml.replace(
        "train:\n  cmd:",
        "train:\n  params:\n  - params.yaml:\n    - model.learning_rate\n    - model.n_estimators\n  cmd:"
    )
    (ROOT / "dvc.yaml").write_text(dvc_yaml, encoding="utf-8")

print((ROOT / "dvc.yaml").read_text(encoding="utf-8")[:400] + " ...")


stages:
  prepare:
    cmd: python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/
    deps:
      - src/data/make_dataset.py
      - data/raw/UCI_Credit_Card.csv
    outs:
      - data/processed/train.csv
      - data/processed/test.csv

  train:
    cmd: python src/models/train.py data/processed/train.csv data/processed/test.csv models/
    deps:
      - src/models/train.py ...


In [32]:
sh("dvc repro")
print((ROOT / "models" / "metrics.json").read_text(encoding="utf-8"))


> dvc repro 
 'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

{
  "model": "GradientBoostingClassifier",
  "model_params": {
    "learning_rate": 0.1,
    "n_estimators": 150
  },
  "roc_auc": 0.7705572956671517,
  "precision": 0.6647230320699709,
  "recall": 0.3436322532027129,
  "f1": 0.45305514157973176
}


In [34]:
# создаю папку-хранилище
(remote_dir := (ROOT / "dvcstore")).mkdir(exist_ok=True)
# добавляю remote (если ещё не добавлен)
sh("dvc remote add -d localstore ./dvcstore || echo already")
# пушу артефакты (модель, кэш датасета и т.д.) в локальное хранилище
sh("dvc push")


> dvc remote add -d localstore ./dvcstore || echo already 
 Setting 'localstore' as a default remote.

> dvc push 
 1 file pushed



CompletedProcess(args='dvc push', returncode=0, stdout='1 file pushed\n', stderr='')

In [36]:
# гоняю 3 эксперимента с разными гиперами (без ручного коммита)
sh("dvc exp run -S model.learning_rate=0.08 -S model.n_estimators=200")
sh("dvc exp run -S model.learning_rate=0.12 -S model.n_estimators=120")
sh("dvc exp run -S model.learning_rate=0.07 -S model.n_estimators=300")

# показываю таблицу: метрики + параметры (оставляю только интересное)
sh("dvc exp show --no-timestamp --only-changed "
   "--include-params model.learning_rate,model.n_estimators "
   "--include-metrics roc_auc,precision,recall,f1")


> dvc exp run -S model.learning_rate=0.08 -S model.n_estimators=200 
 Reproducing experiment 'adunc-veil'
'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

> dvc exp run -S model.learning_rate=0.12 -S model.n_estimators=120 
 Reproducing experiment 'pushy-snip'
'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

> dvc exp run -S model.learning_rate=0.07 -S model.n_estimators=300 
 Reproducing experiment 'fifty-envy'
'data\raw\UCI_Credit_Card.csv.dvc' didn't change, skipping
Running stage 'prepare':
> python src/data/make_dataset.py data/raw/UCI_Credit_Card.csv data/processed/

> dvc exp show --no-timestamp --only-changed --include-params model.learning_rate,model.n_estimators --include-metrics roc_auc,precision,recall,f1 
 usage: dvc experiments show [-h] [-q | -v

CompletedProcess(args='dvc exp show --no-timestamp --only-changed --include-params model.learning_rate,model.n_estimators --include-metrics roc_auc,precision,recall,f1', returncode=254, stdout="usage: dvc experiments show [-h] [-q | -v] [-A] [--rev <commit>] [-n <num>]\n                            [-a] [-T] [--no-pager] [--only-changed]\n                            [--drop <regex_pattern>] [--keep <regex_pattern>]\n                            [--param-deps] [--sort-by <metric/param>]\n                            [--sort-order {asc,desc}] [--sha] [--hide-failed]\n                            [--hide-queued] [--hide-workspace] [--json]\n                            [--csv] [--md] [--precision <n>] [-f]\n\nPrint experiments.\nDocumentation: <https://man.dvc.org/exp/show>\n\noptions:\n  -h, --help            show this help message and exit\n  -q, --quiet           Be quiet.\n  -v, --verbose         Be verbose.\n  -A, --all-commits     Show all experiments in the repository (overrides\n      